## 1. Importando librerias

In [483]:
# DataFrame
import pandas as pd

# Matplot
import matplotlib.pyplot as plt
%matplotlib inline

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer

# Word2vec
import gensim

# Utility
import re
import numpy as np
import os
from collections import Counter
import logging
import time
import pickle
import itertools

# Set log
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [484]:
df = pd.read_csv('../files/reddit_mobile_operators_peru_opinions_dataset.csv', header=0)
df = df.dropna(subset=['comment'])

In [485]:
print("Tamaño del dataset:  ", len(df))

Tamaño del dataset:   5985


In [486]:
#target_cnt = Counter(df.sentiment)
#plt.figure(figsize=(16,8))
#plt.bar(target_cnt.keys(), target_cnt.values())
#plt.title("Distribución de las etiquetas de los conjuntos de datos")

Pre procesando el dataset

In [487]:
# DATASET
TRAIN_SIZE = 0.8

# TEXT CLENAING
TEXT_CLEANING_RE = "http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"

In [488]:
import string

stop_words = ['a','actualmente','adelante','además','afirmó','agregó','ahora','ahí','al','algo','alguna','algunas','alguno','algunos','algún','alrededor','ambos','ampleamos','ante','anterior','antes','apenas','aproximadamente','aquel','aquellas','aquellos','aqui','aquí','arriba','aseguró','así','atras','aunque','ayer','añadió','aún','bajo','bastante','bien','buen','buena','buenas','bueno','buenos','cada','casi','cerca','cierta','ciertas','cierto','ciertos','cinco','comentó','como','con','conocer','conseguimos','conseguir','considera','consideró','consigo','consigue','consiguen','consigues','contra','cosas','creo','cual','cuales','cualquier','cuando','cuanto','cuatro','cuenta','cómo','da','dado','dan','dar','de','debe','deben','debido','decir','dejó','del','demás','dentro','desde','después','dice','dicen','dicho','dieron','diferente','diferentes','dijeron','dijo','dio','donde','dos','durante','e','ejemplo','el','ella','ellas','ello','ellos','embargo','empleais','emplean','emplear','empleas','empleo','en','encima','encuentra','entonces','entre','era','erais','eramos','eran','eras','eres','es','esa','esas','ese','eso','esos','esta','estaba','estabais','estaban','estabas','estad','estada','estadas','estado','estados','estais','estamos','estan','estando','estar','estaremos','estará','estarán','estarás','estaré','estaréis','estaría','estaríais','estaríamos','estarían','estarías','estas','este','estemos','esto','estos','estoy','estuve','estuviera','estuvierais','estuvieran','estuvieras','estuvieron','estuviese','estuvieseis','estuviesen','estuvieses','estuvimos','estuviste','estuvisteis','estuviéramos','estuviésemos','estuvo','está','estábamos','estáis','están','estás','esté','estéis','estén','estés','ex','existe','existen','explicó','expresó','fin','fue','fuera','fuerais','fueran','fueras','fueron','fuese','fueseis','fuesen','fueses','fui','fuimos','fuiste','fuisteis','fuéramos','fuésemos','gran','grandes','gueno','ha','haber','habida','habidas','habido','habidos','habiendo','habremos','habrá','habrán','habrás','habré','habréis','habría','habríais','habríamos','habrían','habrías','habéis','había','habíais','habíamos','habían','habías','hace','haceis','hacemos','hacen','hacer','hacerlo','haces','hacia','haciendo','hago','han','has','hasta','hay','haya','hayamos','hayan','hayas','hayáis','he','hecho','hemos','hicieron','hizo','hoy','hube','hubiera','hubierais','hubieran','hubieras','hubieron','hubiese','hubieseis','hubiesen','hubieses','hubimos','hubiste','hubisteis','hubiéramos','hubiésemos','hubo','igual','incluso','indicó','informó','intenta','intentais','intentamos','intentan','intentar','intentas','intento','ir','junto','la','lado','largo','las','le','les','llegó','lleva','llevar','lo','los','luego','lugar','manera','manifestó','mayor','me','mediante','mejor','mencionó','menos','mi','mientras','mio','mis','misma','mismas','mismo','mismos','modo','momento','mucha','muchas','mucho','muchos','muy','más','mí','mía','mías','mío','míos','nada','nadie','ni','ninguna','ningunas','ninguno','ningunos','ningún','no','nos','nosotras','nosotros','nuestra','nuestras','nuestro','nuestros','nueva','nuevas','nuevo','nuevos','nunca','o','ocho','os','otra','otras','otro','otros','para','parece','parte','partir','pasada','pasado','pero','pesar','poca','pocas','poco','pocos','podeis','podemos','poder','podria','podriais','podriamos','podrian','podrias','podrá','podrán','podría','podrían','poner','por','por qué','porque','posible','primer','primera','primero','primeros','principalmente','propia','propias','propio','propios','próximo','próximos','pudo','pueda','puede','pueden','puedo','pues','que','quedó','queremos','quien','quienes','quiere','quién','qué','realizado','realizar','realizó','respecto','sabe','sabeis','sabemos','saben','saber','sabes','se','sea','seamos','sean','seas','segunda','segundo','según','seis','ser','seremos','será','serán','serás','seré','seréis','sería','seríais','seríamos','serían','serías','seáis','señaló','si','sido','siempre','siendo','siete','sigue','siguiente','sin','sino','sobre','sois','sola','solamente','solas','solo','solos','somos','son','soy','su','sus','suya','suyas','suyo','suyos','sí','sólo','tal','también','tampoco','tan','tanto','te','tendremos','tendrá','tendrán','tendrás','tendré','tendréis','tendría','tendríais','tendríamos','tendrían','tendrías','tened','teneis','tenemos','tener','tenga','tengamos','tengan','tengas','tengo','tengáis','tenida','tenidas','tenido','tenidos','teniendo','tenéis','tenía','teníais','teníamos','tenían','tenías','tercera','ti','tiempo','tiene','tienen','tienes','toda','todas','todavía','todo','todos','total','trabaja','trabajais','trabajamos','trabajan','trabajar','trabajas','trabajo','tras','trata','través','tres','tu','tus','tuve','tuviera','tuvierais','tuvieran','tuvieras','tuvieron','tuviese','tuvieseis','tuviesen','tuvieses','tuvimos','tuviste','tuvisteis','tuviéramos','tuviésemos','tuvo','tuya','tuyas','tuyo','tuyos','tú','ultimo','un','una','unas','uno','unos','usa','usais','usamos','usan','usar','usas','uso','usted','va','vais','valor','vamos','van','varias','varios','vaya','veces','ver','verdad','verdadera','VERDADERO','vez','vosotras','vosotros','voy','vuestra','vuestras','vuestro','vuestros','y','ya','yo','él','éramos','ésta','éstas','éste','éstos','última','últimas','último','últimos','sintiendo','sentido','sentida','sentidos','sentidas','siente','sentid',]
#stop_words = stopwords.words("spanish")
stemmer = SnowballStemmer("spanish")

def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', str(text).lower()).strip()
    text = re.sub('@[^\s]+', ' ', text)
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.translate(str.maketrans('', '', '1234567890'))
    text = re.sub(' +', ' ', text)
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)
#
#print('hola' in stop_words)

In [489]:
#def preprocess(text, stem=False):
#    # Remove link,user and special characters
#    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
#    tokens = []
#    for token in text.split():
#        if token not in stop_words:
#            tokens.append(token)
#            #if stem:
#            #    tokens.append(stemmer.stem(token))
#            #else:
#            #    tokens.append(token)
#    return " ".join(tokens)

In [490]:

%%time
df['comments_preprocess'] = df.comment.apply(lambda x: preprocess(x))

CPU times: total: 812 ms
Wall time: 1.05 s


Dividir la formación y las pruebas

In [491]:
df_train, df_test = train_test_split(df, test_size=1-TRAIN_SIZE, random_state=42)
print("TRAIN size:", len(df_train))
print("TEST size:", len(df_test))

TRAIN size: 4788
TEST size: 1197


Word2Vec

In [492]:
# Asigna df_train.txt a 'documentos'
documentos = df_train.comments_preprocess

# Elimina los duplicados
documentos.drop_duplicates(inplace=True)

# Aplica 'split' a cada documento
documentos = [texto.split() for texto in documentos]

In [493]:
## WORD2VEC 
#W2V_SIZE = 300
#W2V_WINDOW = 15
#W2V_EPOCH = 32
#W2V_MIN_COUNT = 6

# WORD2VEC 
W2V_SIZE = 400
W2V_WINDOW = 12
W2V_EPOCH = 75
W2V_MIN_COUNT = 10

#
#
#w2v_model = gensim.models.word2vec.Word2Vec(vector_size=W2V_SIZE, 
#                                            window=W2V_WINDOW, 
#                                            min_count=W2V_MIN_COUNT, 
#                                            workers=8)
import multiprocessing
hhgroups2vec = gensim.models.word2vec.Word2Vec(
    documentos,
    sg=1,
    seed=1,
    workers=multiprocessing.cpu_count(),
    vector_size=W2V_SIZE,
    min_count=W2V_MIN_COUNT,
    window=W2V_WINDOW,    epochs=20
)


2023-11-15 11:54:17,622 : INFO : collecting all words and their counts
2023-11-15 11:54:17,623 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-11-15 11:54:17,639 : INFO : collected 16736 word types from a corpus of 70708 raw words and 4627 sentences
2023-11-15 11:54:17,640 : INFO : Creating a fresh vocabulary
2023-11-15 11:54:17,648 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=10 retains 1405 unique words (8.40% of original 16736, drops 15331)', 'datetime': '2023-11-15T11:54:17.648147', 'gensim': '4.3.2', 'python': '3.11.6 (tags/v3.11.6:8b6ee5b, Oct  2 2023, 14:57:12) [MSC v.1935 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.23585-SP0', 'event': 'prepare_vocab'}
2023-11-15 11:54:17,649 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=10 leaves 39784 word corpus (56.27% of original 70708, drops 30924)', 'datetime': '2023-11-15T11:54:17.649151', 'gensim': '4.3.2', 'python': '3.11.6 (tags/v3.11.6:8b6ee5b, Oct  2 2023, 14:57:

In [494]:
hhgroups2vec.build_vocab(documentos)

2023-11-15 11:54:20,246 : INFO : collecting all words and their counts
2023-11-15 11:54:20,247 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-11-15 11:54:20,264 : INFO : collected 16736 word types from a corpus of 70708 raw words and 4627 sentences
2023-11-15 11:54:20,265 : INFO : Creating a fresh vocabulary
2023-11-15 11:54:20,272 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=10 retains 1405 unique words (8.40% of original 16736, drops 15331)', 'datetime': '2023-11-15T11:54:20.272066', 'gensim': '4.3.2', 'python': '3.11.6 (tags/v3.11.6:8b6ee5b, Oct  2 2023, 14:57:12) [MSC v.1935 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.23585-SP0', 'event': 'prepare_vocab'}
2023-11-15 11:54:20,273 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=10 leaves 39784 word corpus (56.27% of original 70708, drops 30924)', 'datetime': '2023-11-15T11:54:20.273065', 'gensim': '4.3.2', 'python': '3.11.6 (tags/v3.11.6:8b6ee5b, Oct  2 2023, 14:57:

In [495]:
words = list(hhgroups2vec.wv.key_to_index)
vocab_size = len(words)
print("Vocab size", vocab_size)

Vocab size 1405


In [496]:
%%time
hhgroups2vec.train(documentos, total_examples=len(documentos), epochs=W2V_EPOCH)

2023-11-15 11:54:20,317 : WARNING : Effective 'alpha' higher than previous training cycles
2023-11-15 11:54:20,318 : INFO : Word2Vec lifecycle event {'msg': 'training model with 12 workers on 1405 vocabulary and 400 features, using sg=1 hs=0 sample=0.001 negative=5 window=12 shrink_windows=True', 'datetime': '2023-11-15T11:54:20.318457', 'gensim': '4.3.2', 'python': '3.11.6 (tags/v3.11.6:8b6ee5b, Oct  2 2023, 14:57:12) [MSC v.1935 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.23585-SP0', 'event': 'train'}
2023-11-15 11:54:20,465 : INFO : EPOCH 0: training on 70708 raw words (37267 effective words) took 0.1s, 275945 effective words/s
2023-11-15 11:54:20,590 : INFO : EPOCH 1: training on 70708 raw words (37168 effective words) took 0.1s, 318896 effective words/s
2023-11-15 11:54:20,714 : INFO : EPOCH 2: training on 70708 raw words (37195 effective words) took 0.1s, 321835 effective words/s
2023-11-15 11:54:20,841 : INFO : EPOCH 3: training on 70708 raw words (37185 effective words) took

CPU times: total: 33 s
Wall time: 9.31 s


(2790461, 5303100)

In [497]:
hhgroups2vec.wv.most_similar("excelente")

[('muchísimo', 0.321259081363678),
 ('principal', 0.29011350870132446),
 ('atencion', 0.288583904504776),
 ('momentos', 0.2787383198738098),
 ('madre', 0.2775704860687256),
 ('xddd', 0.27683714032173157),
 ('español', 0.27352607250213623),
 ('soporte', 0.2635883092880249),
 ('dios', 0.26189321279525757),
 ('estabilidad', 0.2603915333747864)]

In [498]:
hhgroups2vec.wv.most_similar("bien")

KeyError: "Key 'bien' not present in vocabulary"

In [ ]:
hhgroups2vec.wv.most_similar("bien")

KeyError: "Key 'bien' not present in vocabulary"

In [ ]:
hhgroups2vec.wv.most_similar("excelente")

[('inmediato', 0.39453625679016113),
 ('veas', 0.37086236476898193),
 ('oro', 0.359712153673172),
 ('simétrica', 0.3575632870197296),
 ('contraté', 0.34624508023262024),
 ('dijiste', 0.3446570932865143),
 ('crecimiento', 0.3371012806892395),
 ('robistar', 0.3337101340293884),
 ('proyectos', 0.3298235833644867),
 ('comunidades', 0.3270324468612671)]